## Демонстрация иерархического с энкодером BERT и плоского классификатора на основе BERT энкодера.

Импортируем необходимые библиотеки и модули, в том числе, модули мпровизированной HierarchicalLibrary, в которых содержатся необходимые для работы иерархического классификатора классы.

In [84]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.preprocessing import LabelEncoder

from HierarchicalLibrary import Classifier, CategoryTree, TextProcessor
from HierarchicalLibrary.Encoders import LdaEncoder, NavecEncoder, FasttextEncoder, BertEncoder

In [43]:
SEED = 1

# Method for increasing the weight of the first words of title
def word_pyramid(string: str, min_n_words: int, max_n_words: int) -> list:
    result = []
    split = string.split(' ')
    for i in range(min_n_words, max_n_words+1):
        result += split[:i]
    return ' '.join(result)

def prepare_data(full_train_data: pd.DataFrame, seed: int, valid_size: int):
    data_full = full_train_data.sample(frac=1, random_state=seed).copy()
    data_full.drop(['rating', 'feedback_quantity'], axis=1, inplace=True)
    data_full.title = data_full.title.astype('string')
    data_full.short_description = data_full.short_description.astype('string')
    data_full.fillna(value='', inplace=True)
    data_full.name_value_characteristics = data_full.name_value_characteristics.astype('string')
    data_full = data_full.assign(Document=[str(x) + ' ' + str(y) + ' ' + str(z) + ' ' + str(x) for x, y, z in zip(data_full['title'], data_full['short_description'], data_full['name_value_characteristics'])])
    data_full.drop(['title', 'short_description', 'name_value_characteristics'], axis=1, inplace=True)
    data_full.Document = data_full.Document.astype('string')

    data = data_full[:-valid_size].reset_index(drop=True)
    data_valid = data_full[-valid_size:].reset_index(drop=True)
    return data, data_valid

def set_seeds(seed: int):  
    np.random.seed(seed)
    

Загружаем данные

In [44]:
cat_tree_df = pd.read_csv('categories_tree.csv', index_col=0)
full_train_data = pd.read_parquet('train.parquet')

Подготавливаем полный, тренировочный и валидационный датасеты: перемешиваем данные в фрейме, удаляем колонки рейтинга и кол-ва отзывов, корректируем типы данных колонок, заполняем пропущенные значения, текст из колонок 'title', 'short_description' и 'name_value_characteristics' объединяем в колонку "Document", добавляем первые слова из колонки 'title', чтобы увеличить их вес.

In [45]:
set_seeds(SEED)
data, data_valid = prepare_data(full_train_data, seed=SEED, valid_size=4000)
#data = data[:20000]

Для ускорения расчетов, оставим только 50000 записей, иначе, считать будет долго.

In [46]:
data

,id,category_id,Document
0,1181186,12350,Маска Masil для объёма волос 8ml /Корейская ко...
1,304936,12917,Силиконовый дорожный контейнер футляр чехол дл...
2,816714,14125,"Тканевая маска для лица с муцином улитки, 100%..."
3,1437391,11574,Браслеты из бисера Браслеты из бисера. Брасле...
4,1234938,12761,Бальзам HAUTE COUTURE LUXURY BLOND для блондир...
...,...,...,...
279447,564872,11635,"Крем-баттер для рук и тела MS.NAILS, 250 мл Кр..."
279448,1002594,12476,"Цепочка на шею, 50 см Красивые, легкие и очень..."
279449,988538,12302,Обложка на паспорт кожаная Кожаная обложка на ...
279450,1014080,13407,Открытка средняя двойная на татарском языке Р...


### Text Processor

Инициализируем объект энкодера (это класс, который управляет расчетами векторов скрытых представлений текстов, "эмбеддингов")

In [47]:
text_processor = TextProcessor(
    add_stop_words=[',', '.', '', '|', ':', '"', '/', ')', '(', 'a', 'х', '(:', '):', ':(', ':)', 'и']
)

Следующий код читает документы из датафрейма, выполняет токенизацию и лемматизацию средствами пакета natasha, затем, сохраняет леммы в собственную переменную Encoder.texts. Лемматизация выполняется достаточно долго, поэтому сохраняем данные на диск:

In [48]:
text_processor.lemmatize_data(data, document_col='Document', id_col='id')
text_processor.save_lemms_data('50000_set_lemm', directory='Hierarhical_BERT')

Lemmatize: 100%|██████████| 279452/279452 [1:56:28<00:00, 39.99it/s]  


Загружаем леммы с диска:

In [49]:
text_processor.load_lemms_data('50000_set_lemm', directory='Hierarhical_BERT')

#### Энкодер на базе модели BERT

In [82]:
bert_encoder = BertEncoder()
bert_encoder.load_model("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at ./cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


В случае необходимости, считаем и сохраняем матрицу снижения размерности эмбеддингов navec и bert:

In [51]:
bert_encoder.calc_pca(texts=text_processor.texts, sample_size=10000)
bert_encoder.save_pca('PCA_bert.pickle')

Загружаем матрицу для понижения размерности navec эмбеддингов (понижение размерности выполнено для увеличения производительности, если есть желание отключить понижение размерности - можно просто не передавать navec энкодеру значение dim):

In [52]:
bert_encoder.load_pca('PCA_bert.pickle')

Размерность эмбеддинга BERT:


In [83]:
bert_encoder.transform([['foo']]).shape[1]

312

#### Словарь

Используя встроенный метод энкодера, формируем словарь эмбеддингов товаров вида {good_id(int) : embedding(np.array)}. 

In [54]:
encoders=[bert_encoder]

In [55]:
embeddings_dict = text_processor.make_embeddings_dict(encoders=encoders)

Сохраняем словарь эмбеддингов при необходимости - загружаем сохранённый:

In [56]:
path = os.path.join(Path(".").parent, 'Hierarhical_BERT', '50000_set_embs_dict.pickle')
with open(path, 'wb') as f:
    pickle.dump(embeddings_dict, f)

In [57]:
path = os.path.join(Path(".").parent, 'Hierarhical_BERT', '50000_set_embs_dict.pickle')
with open(path, 'rb') as f:
    embeddings_dict = pickle.load(f)

### Дерево каталога

Инициализируем дерево каталога - CategoryTree() - это класс, который хранит все узлы, необходимую информацию для обучения, а также реализует алгоритмы заполнения дерева, обхода при инференсе для определения категории товара. 
Добавляем узлы из таблицы categories_tree.csv, затем, добавляем товары из тренировочной выборки.

In [58]:
cat_tree = CategoryTree()
cat_tree.add_nodes_from_df(cat_tree_df, parent_id_col='parent_id', title_col='title')
cat_tree.add_goods_from_df(data, category_id_col='category_id', good_id_col='id')

Записываем эмбеддинги в дерево каталогов (производится расчет эмбеддингов узлов как усреднённых эмбеддингов документов, попавших в каждый узел):

In [59]:
cat_tree.update_embeddings(embeddings_dict)

Примешиваем к эмбеддингам узлов эмбеддинги их собственных описаний.

In [60]:
cat_tree.mix_in_description_embs(lambda titles: text_processor.get_embeddings(titles, encoders=encoders), weight=10)

### Классификатор

Инициализируем объект классификатора - он управляет процессом получения вероятностей принадлежности товара к узлу (predict_proba). CatBoost больше не нужен - просто небудем его инициализировать..

In [61]:
classifier = Classifier(tol=0.03, max_iter=300)

Обучаем локальные веса (модель логистической регрессии в каждом из узлов дерева). Сохраняем дерево (так как считается очень долго). При необходимости - загружаем.

In [62]:
cat_tree.fit_local_weights(classifier, embeddings_dict, C=0.05, reg_count_power=0.5)

100%|██████████| 3370/3370 [1:04:40<00:00,  1.15s/it]


Сохраняем, и при необходимости, загружаем дерево с рассчитанными весами.

In [63]:
cat_tree.save_tree('50000_set_tree.pickle', directory='Hierarhical_BERT')

In [64]:
cat_tree.load_tree('50000_set_tree.pickle', directory='Hierarhical_BERT')

### Тестирование модели

#### Тестирование на трейне

Формируем массив эмбеддингов для тестирования

In [65]:
begin_example = 0
end_example = 3000
train_documents = data.Document.tolist()[begin_example:end_example]
train_target = data.category_id.tolist()[begin_example:end_example]
embs = text_processor.get_embeddings(train_documents, encoders=encoders)

Выполняем поиск категорий по каталогу для каждого тестового примера

In [66]:
pred_leafs = []
for i in tqdm.tqdm(range(len(embs)), total=len(embs)):
    pred_leafs.append(cat_tree.choose_leaf(classifier = classifier, good_embedding=embs[i]))

100%|██████████| 3000/3000 [00:12<00:00, 247.28it/s]


In [67]:
print(f'Train set hF1={cat_tree.hF1_score(train_target, pred_leafs):.3f}') 

Train set hF1=0.729


#### Тестирование на отложенной выборке

Формируем массив эмбеддингов для тестирования

In [68]:
begin_example = 0
end_example = 4000
valid_documents = data_valid.Document.tolist()[begin_example:end_example]
valid_target = data_valid.category_id.tolist()[begin_example:end_example]
embs_valid = text_processor.get_embeddings(valid_documents, encoders=encoders)

Выполняем поиск категорий по каталогу для каждого тестового примера

In [69]:
pred_leafs_valid = []
for i in tqdm.tqdm(range(len(embs_valid)), total=len(embs_valid)):
    pred_leafs_valid.append(cat_tree.choose_leaf(classifier = classifier, good_embedding=embs_valid[i]))

100%|██████████| 4000/4000 [00:16<00:00, 245.99it/s]


In [70]:
print(f'Validation hF1={cat_tree.hF1_score(valid_target, pred_leafs_valid):.3f}') 

Validation hF1=0.718


В этом ноутбуке гиперпараметры и размер выборки выбраны такими, чтобы расчёты выполнялись относительно быстро. С хорошими гиперпараметрами, на полном размере выборки, удалось получить hF1=0.86, что значительно ниже бейзлайна. 

## PyTorch

In [71]:
class KEdataset(Dataset):
    def __init__(self, data: pd.DataFrame, 
                 embeddings_dict: dict = None,
                 document_list: list = None,
                 encoders = None,
                 id_col: str = 'id', 
                 category_col: str = None, 
                 document_col = None,
                 text_processor: object = None,
                 mode: str = 'test',
                 label_encoder: object = None,
                 simple_lemms: bool = False) -> None:
        super().__init__()
        
        if embeddings_dict:
            self.X = torch.tensor(np.array(
                data[id_col].apply(lambda good_id: embeddings_dict[good_id]).tolist()), dtype=torch.float32)
        else:
            self.X = torch.tensor(text_processor.get_embeddings(
                data[document_col].tolist(), 
                encoders=encoders, 
                simple_lemms=simple_lemms), dtype=torch.float32)
        
        self.mode = mode

        if self.mode not in ['train', 'val', 'test']:
            print(f"{self.mode} is not correct; correct modes: {['train', 'val', 'test']}")
            raise NameError
            
        if label_encoder:
            self.label_encoder = label_encoder
        else:
            self.label_encoder = LabelEncoder()

        if self.mode == 'train':
            self.labels = data[category_col].tolist()
            self.label_encoder = LabelEncoder()
            self.label_encoder.fit(self.labels)
        elif self.mode == 'val':
            self.labels = data[category_col].tolist()
            self.label_encoder = label_encoder            
        return
        
    def __len__(self):
        return self.X.shape[0]
  
    def __getitem__(self, index):
        
        x = self.X[index]

        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
    
    @property
    def dim(self):
        return train_dataset.X.shape[1]
        

In [72]:
train_dataset = KEdataset(data=data, 
                          embeddings_dict=embeddings_dict, 
                          id_col='id', 
                          category_col='category_id', 
                          mode='train')

valid_dataset = KEdataset(data=data_valid, 
                          encoders=encoders, 
                          document_col='Document', 
                          category_col='category_id', 
                          mode='val', 
                          text_processor=text_processor,
                          label_encoder=train_dataset.label_encoder, 
                          simple_lemms=True)


In [73]:
class Fcnn(nn.Module):
  
    def __init__(self, emb_dim: int, n_classes: int):
        super().__init__()
        self.out = nn.Linear(emb_dim, n_classes)
        return
  
    def forward(self, x):
        logits = self.out(x)
        return logits

In [74]:
def fit_epoch(model, train_loader, criterion, optimizer, sheduler, device: str = 'cpu'):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        inputs = inputs.to(torch.device(device))
        labels = labels.to(torch.device(device))
        optimizer.zero_grad()

        outputs = model(inputs)
        #target = F.one_hot(labels, 1231).float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
    sheduler.step()
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

def eval_epoch(model, val_loader, criterion, device: str = 'cpu'):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    val_preds = []
    val_labels = []

    for inputs, labels in val_loader:
        inputs = inputs.to(torch.device(device))
        labels = labels.to(torch.device(device))

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            #target = F.one_hot(labels, 1231).float()
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    val_preds = []
    val_labels = []
    return val_loss, val_acc

def train(train_dataset, 
          val_dataset, 
          model, epochs, 
          batch_size, 
          num_workers=0, 
          lr: float = 0.01, lr_mult: float = 0.1,
          weight_decay=0.0):
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm.tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        gamma = lr_mult ** (2/epochs)
        sheduler = lr_scheduler.StepLR(opt, step_size=2, gamma=gamma, verbose=True)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, sheduler)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            pbar_outer.update(1)
            tqdm.tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
    return history

def predict(model, test_loader, device: str = 'cpu'):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(torch.device(device))
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

Инициализация сети:

In [75]:
n_classes = len(np.unique(data.category_id.values))
simple_nn = Fcnn(n_classes=n_classes, 
                  emb_dim=train_dataset.dim).to(torch.device("cpu"))

print("we will classify :{}".format(n_classes))
print(simple_nn)

we will classify :1231
Fcnn(
  (out): Linear(in_features=312, out_features=1231, bias=True)
)


Обучение сети:

In [81]:
history = train(train_dataset, 
                 valid_dataset, 
                 model=simple_nn, 
                 epochs=20, 
                 batch_size=18630, 
                 num_workers=0, 
                 lr=0.01, lr_mult = 0.03, 
                 weight_decay=0.1) 
#

epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-02.
Adjusting learning rate of group 0 to 1.0000e-02.
loss 0.8535139397301361


epoch:   5%|▌         | 1/20 [01:08<21:44, 68.68s/it]


Epoch 001 train_loss: 0.8535     val_loss 1.5638 train_acc 0.8181 val_acc 0.6623
Adjusting learning rate of group 0 to 7.0423e-03.
loss 0.7585561316502336


epoch:  10%|█         | 2/20 [02:15<20:20, 67.82s/it]

...


Epoch 004 train_loss: 0.6983     val_loss 1.5429 train_acc 0.8427 val_acc 0.6567
Adjusting learning rate of group 0 to 4.9593e-03.
loss 0.6817098708134846


epoch:  25%|██▌       | 5/20 [05:37<16:49, 67.29s/it]


Epoch 005 train_loss: 0.6817     val_loss 1.5564 train_acc 0.8459 val_acc 0.6520


epoch:  25%|██▌       | 5/20 [06:11<18:33, 74.23s/it]


KeyboardInterrupt: 

Тест:

In [77]:
test_dataset = KEdataset(data=data_valid, 
                         encoders=encoders, 
                         document_col='Document', 
                         category_col='category_id', 
                         mode='test', 
                         text_processor=text_processor,
                         label_encoder=train_dataset.label_encoder, 
                         simple_lemms=True)
val_preds = predict(simple_nn, 
                    DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
                    )

In [78]:
pred_torch_valid = list(train_dataset.label_encoder.inverse_transform(val_preds.argmax(axis=1)))
print(f'Validation hF1={cat_tree.hF1_score(valid_target, pred_torch_valid):.3f}')

Validation hF1=0.741
